# Détectection des faux billets avec Python

## 1. Preliminary

     *** Contexte ***

In [ ]:
cd ..

Identification des contrefaçons des billets en euros

Les billets d'euro ont des valeurs nominales de 5, 10, 20, 50, 100, 200 et 500 euros. 

In [ ]:
!python -V

## 1.1 Install

In [ ]:
#!pip install --upgrade seaborn
#!pip install --upgrade pandas
#!pip install pandas --user
#!pip install --upgrade numexpr
#!pip install --upgrade bottleneck

#!pip install --upgrade matplotlib
#!pip install matplotlib --user

# 1.2 Importation

## 1.2.1 Importation des librairies

In [ ]:
#builtin
import os

In [ ]:

#data
import pandas as pd
import numpy as np

In [ ]:

#visualisation 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import missingno as msno
from sklearn.model_selection import LearningCurveDisplay
%matplotlib inline

In [ ]:
#machine learning
    #estimation
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.dummy import DummyRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import LearningCurveDisplay, ShuffleSplit
from sklearn.model_selection import train_test_split
    #standardization
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

## 2.1 chargement des fichiers

In [ ]:
input_folder = r"C:\Users\Nada\Desktop\OpenClassRooms\Projet 10\Data\0 Source"

input_folder = "./data/source/"
# Read CSV train data file into DataFrame
train_df= pd.read_csv(os.path.join(input_folder, "billets.csv"), dtype=float, sep=';')
# Read CSV test data file into DataFrame
test_df = pd.read_csv(os.path.join(input_folder, "billets_production.csv"), sep=',')

## 2. Prétraitement des données

## 2.2 Exploration

In [ ]:
#Affichage des 5 premieres lignes
train_df.head()

In [ ]:
print('Le nombre des itemes dans le DataFrame train_df est {}.'.format(train_df.shape[0]))

In [ ]:
# preview test data
test_df

test DataFrame contient 5 lignes sans valeurs manquantes.

In [ ]:
print('Le nombre des itemes dans le DataFrame train_df est {}.'.format(test_df.shape[0]))

Note: On ne voit pas la colonne 'is_genuine' (la target) dans le dataset test_df. Notre objectif est alors de prédire la target par differentes algorithmes de machine learning comme la regression logistique.

In [ ]:
#Affichage des 5 dernieres lignes
train_df.tail()

In [ ]:
#Affichage de 5 lignes arbitrairement
train_df.sample(5)

In [ ]:
#Dimensions du DataFrame
train_df.shape

In [ ]:
#Information sur les colonnes
train_df.dtypes

In [ ]:
#Statistiques descriptives
train_df.describe().round(2)

In [ ]:
#Nombre des doublons dans le DataFrame
train_df.duplicated().sum()

In [ ]:
#Nombre des doublons sans le target
train_df.drop(columns="is_genuine").duplicated().sum()

In [ ]:
#Information sur les valeurs unique dans chaque colonne
train_df.nunique()

In [ ]:
#Nombre des valeurs dans la target
train_df['is_genuine'].value_counts()

In [ ]:
#creation de la colonne target
train_df.rename(columns={'is_genuine':'target'}, inplace=True)

In [ ]:
data=train_df['target'].value_counts()
labels=data.index
plt.pie(data, labels=labels, autopct='%1.1f%%')
plt.title('Nombre des vrais et faux billets')
plt.show()

1=Vrai
0=faux

## 2.3 Nettoyage des données

In [ ]:
#Nombre des valeurs manquantes dans chaque colonne
train_df.isnull().sum()

In [ ]:
#Nombre des valeurs manquantes dans chaque colonne
tmp = train_df.isnull().mean()
tmp[tmp > 0]

On a 37 valeurs manquantes dans la colonne margin_low

In [ ]:
# Pourçentage des valeurs manquantes 
print('Percent of missing "margin_low" records is %.2f%%' %((train_df['margin_low'].isnull().sum()/train_df.shape[0])*100))

~2,5% des données dans la colonne margin_low est manquantes. -> voir la distribution de cette variable

In [ ]:
# la distribution de la variable 'margin_low'
ax = train_df["margin_low"].hist(bins=15, density=True, stacked=True, color='teal', alpha=0.6)
train_df["margin_low"].plot(kind='density', color='teal')
ax.set(xlabel='margin_low')
plt.xlim(0,10)
plt.show()

### 2.3.1 Imputation des valeurs manquantes

In [ ]:
train_df

On a un DataFrame qui contient X=6 (features)

## Imputation des valeurs manquantes

## les valeurs manquantes sont de type numerique

*** 1. La methode regression lineaire de sklearn ***

In [ ]:
# Séparer les données en deux ensembles : avec et sans valeurs manquantes

test= train_df[train_df['margin_low'].isna()] #DataFrame qui contient que des valeurs manquantes dans la colonne margin_low

train = train_df[~train_df['margin_low'].isna()] #dataframe sans valeurs manquantes

In [ ]:
test.shape

In [ ]:
train.shape

In [ ]:
#les variables explicative (X)
X_train = train.drop(columns=['margin_low', 'target']) 
X_train.head()

In [ ]:
# Création d'une instance de StandardScaler
scaler_1 = StandardScaler()
# Adapter le scaler aux données
scaler_1.fit(X_train)

# Standardiser les données
X_train_scaled = scaler_1.transform(X_train)
X_train_scaled[:5]

In [ ]:
#la variable cible (y) (target)
y_train= train['margin_low']
y_train.shape

In [ ]:
# Initialiser les modèles
models = {
    'Linear Regression': LinearRegression(),
    'Naive Model (median)': DummyRegressor(strategy='median'),
    'Naive Model (Mean)': DummyRegressor(strategy='mean')
}

# La validation croisée et les scores RMSE pour chaque modèle
results = {}

for model_name, model in models.items():
    scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = (-scores) ** 0.5
    results[model_name] = {'mean_rmse': rmse_scores.mean(), 'std_rmse': rmse_scores.std()}

# Affichage des résultats
for model_name, result in results.items():
    print(f"Model: {model_name}")
    print(f"Mean RMSE: {result['mean_rmse']}")
    print(f"Standard deviation of RMSE: {result['std_rmse']}\n")


Selon les métriques de RMSE (erreur), le modèle de régression linéaire démontre la meilleure performance parmi les trois modèles testés.

In [ ]:
## Model_1: Régression linéaire
# Créer et entraîner le modèle de régression linéaire
model_1 = LinearRegression()
model_1.fit(X_train_scaled, y_train)

In [ ]:
#creation de la variable X_test
X_test = test.drop(columns=['margin_low', 'target'])

In [ ]:
X_test.head()

In [ ]:
# Standardisation des données de X_test
X_test_scaled = scaler_1.transform(X_test)
X_test_scaled[:5]

In [ ]:
# Prédire les valeurs manquantes
y_predicted = model_1.predict(X_test_scaled)

In [ ]:
y_predicted.shape

In [ ]:
y_predicted

In [ ]:
#Creation d'une copy de notre DataFrame
df_reg=train_df.copy()

In [ ]:
# Remplacement des valeurs manquantes par les valeurs prédites
df_reg.loc[df_reg['margin_low'].isna(), 'margin_low'] = y_predicted

In [ ]:
#Verification
df_reg.isnull().sum()

In [ ]:
# preview adjusted train data
df_reg.head()

In [ ]:
#Verification avec visualisation
plt.figure(figsize=(15,8))
ax = train_df["margin_low"].hist(bins=15, density=True, stacked=True, color='teal', alpha=0.6)
train_df["margin_low"].plot(kind='density', color='teal')
ax = df_reg["margin_low"].hist(bins=15, density=True, stacked=True, color='orange', alpha=0.5)
df_reg["margin_low"].plot(kind='density', color='orange')
ax.legend(['Raw margin_low', 'Adjusted margin_low'])
ax.set(xlabel='margin_low')
plt.xlim(0,10)
plt.show()

2. La méthode de régression linéaire de Statsmodels

In [ ]:
# Créer et ajuster le modèle aux données d'entraînement
model_2 = sm.OLS(y_train, X_train_scaled)
model_fit = model_2.fit()

# Faire des prédictions sur les données de test mises à l'échelle
predicted_values = model_fit.predict(X_test_scaled)

# Afficher les valeurs prédites
print(predicted_values)

In [ ]:
#On a besoin d'une constante dans ce maodéle
X_train_scaled_with_const = sm.add_constant(X_train_scaled)
X_test_scaled_with_const = sm.add_constant(X_test_scaled)
X_test_scaled_with_const[:5]

In [ ]:
model_2 = sm.OLS(y_train, X_train_scaled_with_const).fit()

In [ ]:
model_2.params

In [ ]:
model_2.summary()

In [ ]:
y_train.mean()

In [ ]:
predicted_values = model_2.predict(X_test_scaled_with_const)

In [ ]:
predicted_values

In [ ]:
df_stat=train_df.copy()

In [ ]:
df_stat.loc[df_stat['margin_low'].isnull(), 'margin_low'] = predicted_values

In [ ]:
#Verification avec visualisation
plt.figure(figsize=(15,8))
ax = train_df["margin_low"].hist(bins=15, density=True, stacked=True, color='teal', alpha=0.6)
train_df["margin_low"].plot(kind='density', color='teal')
ax = df_stat["margin_low"].hist(bins=15, density=True, stacked=True, color='orange', alpha=0.5)
df_stat["margin_low"].plot(kind='density', color='orange')
ax.legend(['Raw margin_low', 'Adjusted margin_low'])
ax.set(xlabel='margin_low')
plt.xlim(0,10)
plt.show()

***Exploration des données***

In [ ]:
sns.pairplot(df_reg, hue='target', corner=True)
plt.show()

In [ ]:
correlation_matrix = df_reg.corr()
masque=np.triu(correlation_matrix)
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", mask=masque, vmin=-1, vmax=1)
plt.show()

la target est tres correlée avec la longeur des billets

## 2.3.2 Detection des outliers

In [ ]:
plt.figure(figsize=(20, 20))
sns.set_theme(style="whitegrid")

plt.subplot(321)
sns.boxplot(data=df_reg, x='diagonal')

plt.subplot(322)
sns.boxplot(data=df_reg, x='height_left')

plt.subplot(323)
sns.boxplot(data=df_reg, x='height_right')

plt.subplot(324)
sns.boxplot(data=df_reg, x='margin_low')

plt.subplot(325)
sns.boxplot(data=df_reg, x='margin_up')

plt.subplot(326)
sns.boxplot(data=df_reg, x='length')

plt.show()

Les boxplots révèlent la présence d'outliers dans plusieurs features. <br> Cela suggère que ces valeurs pourraient être liées à des faux billets, caractérisés par des dimensions non conformes, ou bien à des billets de grande valeur, comme les billets de 500€, ou de petites valeurs. <br> Ainsi, il est prévu de conserver les outliers afin de poursuivre l'analyse.

## 2.3.1.1 Data mining

## 2.4 Preparation des données

## Choix du modéle

Type de données categorielles


In [ ]:
df_reg.info()

In [ ]:
df_reg.head()

2.5 Modelisation avec split

2.5.1 Dummy clussifier

In [ ]:
X=df_reg.drop(columns='target')
y=df_reg['target']

In [ ]:
train_X, test_X, train_y, test_y = train_test_split (X, y, test_size=0.3, random_state=42)

In [ ]:
estimator_1= DummyClassifier(strategy="most_frequent")
estimator_1.fit(train_X, train_y)

In [ ]:
y_pred_1 = estimator_1.predict(test_X)
y_pred_1[:5]

In [ ]:
pd.Series(y_pred_1).value_counts()

In [ ]:
test_y.value_counts(normalize=True)

In [ ]:
test_y.value_counts()

In [ ]:
train_X.head()

In [ ]:
tr_score= estimator_1.score(train_X, train_y)
te_score= estimator_1.score(test_X, test_y)
print(f"le score de train est {tr_score}, et de test est {te_score}")

In [ ]:
y_pred_1 = estimator_1.predict(test_X)
y_pred_1[:5]

In [ ]:
test_y.values

In [ ]:
1 - abs(y_pred_1 - test_y.values).mean()

In [ ]:
mat=confusion_matrix(test_y, y_pred_1, labels=estimator_1.classes_)
mat

In [ ]:
test_y.value_counts()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
disp = ConfusionMatrixDisplay(confusion_matrix=mat, display_labels= estimator_1.classes_)
disp.plot()
plt.show()

161 des billets (faux billets) ont ete mal classés

## 2. Regression logistique

In [ ]:
estimator_2 = LogisticRegression(solver='liblinear') #small dataset

In [ ]:
estimator_2.fit(train_X, train_y)

In [ ]:
y_pred_2=estimator_2.predict(test_X)
y_pred_2[:5]

In [ ]:
y_prob = estimator_2.predict_proba(test_X).round(2) #La probabilité de l'appartenance à tel ou tel classe
y_prob[:5]

In [ ]:

conf=confusion_matrix(test_y, y_pred_2, labels=estimator_2.classes_)
conf

In [ ]:

disp = ConfusionMatrixDisplay(confusion_matrix=conf, display_labels= estimator_2.classes_)
disp.plot()
plt.show()

## SVM

In [ ]:
estimator_3 = LinearSVC()

In [ ]:
estimator_3.fit(train_X, train_y)

In [ ]:
y_pred_3=estimator_3.predict(test_X)
y_pred_3[:5]

In [ ]:
conf=confusion_matrix(test_y, y_pred_3, labels=estimator_3.classes_)
conf

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=conf, display_labels= estimator_3.classes_)
disp.plot()
plt.show()

Modelisation avec cross_validation

In [ ]:
X=df_reg.drop(columns=['target'])
y=df_reg['target']

In [ ]:
# Création d'une instance de StandardScaler
scaler_2 = StandardScaler()
# Adapter le scaler aux données
scaler_2.fit(X)

# Standardiser les données
X_scaled = scaler_2.transform(X)
X_scaled[:5]

In [ ]:
#Choix du modele
Logistic_Regression = LogisticRegression()
svc = SVC()
Majority_Model = DummyClassifier(strategy='most_frequent')

# Initialiser les modèles
models = {
    'Logistic Regression': LogisticRegression(),
    'SVC': SVC(),
    'Majority Model': DummyClassifier(strategy='most_frequent')
}

# La validation croisée et les scores de précision pour chaque modèle
results = {}

for model_name, model in models.items():
    scores = cross_val_score(model, X_scaled, y, cv=5, scoring='accuracy')
    results[model_name] = {'mean_accuracy': scores.mean(), 'std_accuracy': scores.std()}

# Affichage des résultats
for model_name, result in results.items():
    print(f"Model: {model_name}")
    print(f"Mean Accuracy: {result['mean_accuracy']}")
    print(f"Standard deviation of Accuracy: {result['std_accuracy']}\n")


In [ ]:
# Évaluation de la performance du modèle

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 6), sharey=True)

common_params = {
    "X": X_scaled,
    "y": y,
    "train_sizes": np.linspace(0.1, 1.0, 5),
    "cv": ShuffleSplit(n_splits=50, test_size=0.2, random_state=0),
    "score_type": "both",
    "n_jobs": 4,
    "line_kw": {"marker": "o"},
    "std_display_style": "fill_between",
    "score_name": "Accuracy",
}

for ax_idx, estimator in enumerate([Logistic_Regression, svc, Majority_Model]):
    LearningCurveDisplay.from_estimator(estimator, **common_params, ax=ax[ax_idx])
    handles, label = ax[ax_idx].get_legend_handles_labels()
    ax[ax_idx].legend(handles[:2], ["Training Score", "Test Score"])
    ax[ax_idx].set_title(f"Learning Curve for {estimator.__class__.__name__}")

En observant les courbes d'apprentissage, on peut déterminer que le modèle DummyClassifier montre un sous-apprentissage (underfitting) tandis que les deux modèles Logistic Regression et SVC sont comparables et montrent de bons résultats. <br> On a choisi d'optimiser le modele de Logistic Regression.

In [ ]:
#Optimisation des hyperparamètres

# Définir la grille des hyperparamètres à rechercher
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

# Initialiser LogisticRegression
logistic_regression = LogisticRegression()

# Créer l'objet GridSearchCV
grid_search = GridSearchCV(logistic_regression, param_grid, cv=5, scoring='accuracy', n_jobs=4, verbose=1, return_train_score=True)

# Exécuter la recherche sur grille sur les données d'entraînement
grid_search.fit(X_scaled, y)

# Afficher les meilleurs hyperparamètres et le score associé
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

In [ ]:
def resultize(grid_search) : 
    """build a DataFrame from the results of a GridSearchCV"""

    res = pd.DataFrame(grid_search.cv_results_)
    cols = [i for i in res.columns if 'split' not in i]
    res = res[cols]
    res = res.round(2).iloc[:, 4:].sort_values('mean_test_score', ascending=False)

    return res

In [ ]:
resultize(grid_search).head(10)

In [ ]:
#Choisir le bon modele
best_model = grid_search.best_estimator_

In [ ]:
#matrice du confusion
y_pred = best_model.predict(X_scaled)
mat=confusion_matrix(y, y_pred)
mat

In [ ]:
pd.DataFrame({"y_true" : y, "y_pred" : y_pred})

In [ ]:
sns.heatmap(mat, annot=True, cmap='coolwarm', fmt='d', cbar=False, vmax=1000, vmin=-1000)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=mat)
disp.plot()
plt.show()

On voit que nous avons 10 faux positifs (10 faux billets classés comme vrais) et 4 faux négatifs (4 vrais billets classés comme faux). <br> Nous allons essayer de changer la métrique de scoring de l'accuracy à l'average_precision.

In [ ]:
# Définir la grille des hyperparamètres à rechercher
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

# Initialiser LogisticRegression
logistic_regression = LogisticRegression()

# Créer l'objet GridSearchCV
grid_search = GridSearchCV(logistic_regression, param_grid, cv=5, scoring='average_precision')

# Exécuter la recherche sur grille sur les données d'entraînement
grid_search.fit(X_scaled, y)

# Afficher les meilleurs hyperparamètres et le score associé
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)


In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
#matrice de confusion
y_pred = best_model.predict(X_scaled)
mat=confusion_matrix(y, y_pred)
mat

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=mat)
disp.plot()
plt.show()